In [6]:
import os

task_type = 'OBJECT_DETECTION'

input_coco_file_path = r"./CustomVisionExport_Coco.json"

output_dir = 'JsonLines'

# default output file name
json_line_file_name = 'LabeledData.jsonl'
output_jsonl_file_path =  os.path.join(os.getcwd(), output_dir, json_line_file_name)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [7]:
import json

class CocoToJSONLinesConverter:
    def convert(self): raise NotImplementedError

class MultiClassConverter(CocoToJSONLinesConverter):
    def __init__(self, coco_data):
        self.json_lines_data = []
        self.categories = {}
        self.coco_data = coco_data
        for i in range(0, len(coco_data['images'])):
            self.json_lines_data.append({})
        for i in range(0, len(coco_data['categories'])):
            self.categories[coco_data['categories'][i]['id']] = coco_data['categories'][i]['name']

    def _populate_image_url(self, json_line, coco_image_url):
        json_line['image_url'] = coco_image_url
        return json_line

    def _populate_label(self, json_line, label_id):
        json_line['label'] = self.categories[label_id]
        return json_line

    def _populate_label_confidence(self, json_line):
        json_line['label_confidence'] = 1.0
        return json_line

    def convert(self):
        for i in range(0, len(self.coco_data['images'])):
            self.json_lines_data[i] = {}
            self.json_lines_data[i] = self._populate_image_url(self.json_lines_data[i], self.coco_data['images'][i]['coco_url'])
            self.json_lines_data[i] = self._populate_label(self.json_lines_data[i], self.coco_data['annotations'][i]['category_id'])
            self.json_lines_data[i] = self._populate_label_confidence(self.json_lines_data[i])
        return self.json_lines_data


class MultiLabelConverter(CocoToJSONLinesConverter):
    def __init__(self, coco_data):
        self.json_lines_data = []
        self.categories = {}
        self.coco_data = coco_data
        self.image_id_to_data_index = {}
        for i in range(0, len(coco_data['images'])):
            self.json_lines_data.append({})
            self.json_lines_data[i]['image_url'] = ""
            self.json_lines_data[i]['label'] = []
            self.json_lines_data[i]['label_confidence'] = []
        for i in range(0, len(coco_data['categories'])):
            self.categories[coco_data['categories'][i]['id']] = coco_data['categories'][i]['name']

    def _populate_image_url(self, index, coco_image):
        self.json_lines_data[index]['image_url'] = coco_image['coco_url']
        self.image_id_to_data_index[coco_image['id']] = index

    def _populate_label(self, annotation):
        index = self.image_id_to_data_index[annotation['image_id']]
        self.json_lines_data[index]['label'].append(self.categories[annotation['category_id']])
        self._populate_label_confidence(index)

    def _populate_label_confidence(self, index):
        self.json_lines_data[index]['label_confidence'].append(1.0)

    def convert(self):
        for i in range(0, len(self.coco_data['images'])):
            self._populate_image_url(i, self.coco_data['images'][i])
        for i in range(0, len(self.coco_data['annotations'])):
            self._populate_label(self.coco_data['annotations'][i])
        return self.json_lines_data


class BoundingBoxConverter(CocoToJSONLinesConverter):
    def __init__(self, coco_data):
        self.json_lines_data = []
        self.categories = {}
        self.coco_data = coco_data
        self.image_id_to_data_index = {}
        for i in range(0, len(coco_data['images'])):
            self.json_lines_data.append({})
            self.json_lines_data[i]['image_url'] = ""
            self.json_lines_data[i]['label'] = []
            self.json_lines_data[i]['label_confidence'] = []
        for i in range(0, len(coco_data['categories'])):
            self.categories[coco_data['categories'][i]['id']] = coco_data['categories'][i]['name']

    def _populate_image_url(self, index, coco_image):
        self.json_lines_data[index]['image_url'] = coco_image['coco_url']
        self.image_id_to_data_index[coco_image['id']] = index

    def _populate_bbox_in_label(self, label, annotation):
        label['topX'] = annotation['bbox'][0]
        label['topY'] = annotation['bbox'][1]
        label['bottomX'] = annotation['bbox'][0] + annotation['bbox'][2]
        label['bottomY'] = annotation['bbox'][1] + annotation['bbox'][3]

    def _populate_label(self, annotation):
        index = self.image_id_to_data_index[annotation['image_id']]
        label = {'label': self.categories[annotation['category_id']]}
        self._populate_bbox_in_label(label, annotation)
        self.json_lines_data[index]['label'].append(label)
        self._populate_label_confidence(index)

    def _populate_label_confidence(self, index):
        self.json_lines_data[index]['label_confidence'].append(1.0)

    def convert(self):
        for i in range(0, len(self.coco_data['images'])):
            self._populate_image_url(i, self.coco_data['images'][i])
        for i in range(0, len(self.coco_data['annotations'])):
            self._populate_label(self.coco_data['annotations'][i])
        return self.json_lines_data


class PolygonConverter(CocoToJSONLinesConverter):
    def __init__(self, coco_data):
        self.json_lines_data = []
        self.categories = {}
        self.coco_data = coco_data
        self.image_id_to_data_index = {}
        for i in range(0, len(coco_data['images'])):
            self.json_lines_data.append({})
            self.json_lines_data[i]['image_url'] = ""
            self.json_lines_data[i]['label'] = []
            self.json_lines_data[i]['label_confidence'] = []
        for i in range(0, len(coco_data['categories'])):
            self.categories[coco_data['categories'][i]['id']] = coco_data['categories'][i]['name']

    def _populate_image_url(self, index, coco_image):
        self.json_lines_data[index]['image_url'] = coco_image['coco_url']
        self.image_id_to_data_index[coco_image['id']] = index

    def _populate_bbox_in_label(self, label, annotation):
        top_x = annotation['bbox'][0]
        top_y = annotation['bbox'][1]
        bottom_x = annotation['bbox'][0] + annotation['bbox'][2]
        bottom_y = annotation['bbox'][1] + annotation['bbox'][3]
        label['bbox'] = [top_x, top_y, bottom_x, bottom_y]

    def _populate_label(self, annotation):
        index = self.image_id_to_data_index[annotation['image_id']]
        label = {'label': self.categories[annotation['category_id']], 'isCrowd': False,
                 'polygon': annotation['segmentation']}
        self._populate_bbox_in_label(label, annotation)
        self.json_lines_data[index]['label'].append(label)
        self._populate_label_confidence(index)

    def _populate_label_confidence(self, index):
        self.json_lines_data[index]['label_confidence'].append(1.0)

    def convert(self):
        for i in range(0, len(self.coco_data['images'])):
            self._populate_image_url(i, self.coco_data['images'][i])
        for i in range(0, len(self.coco_data['annotations'])):
            self._populate_label(self.coco_data['annotations'][i])
        return self.json_lines_data


In [9]:

def read_coco_file(coco_file):
    with open(coco_file) as f_in:
        return json.load(f_in)

def write_json_lines(converter, filename):
    json_lines_data = converter.convert()
    with open(filename, 'w') as outfile:
        for json_line in json_lines_data:
            json.dump(json_line, outfile, separators=(',', ':'))
            outfile.write('\n')

print('AML Data Labeling Task type: {}'.format(task_type))
print('Converting COCO annotation file "{}" into JSON Line file "{}"'.format(input_coco_file_path, output_jsonl_file_path))

coco_data = read_coco_file(input_coco_file_path)

if task_type == 'IMAGE_CLASSIFICATION':
    converter = MultiClassConverter(coco_data)
    write_json_lines(converter, output_jsonl_file_path)
elif task_type == 'IMAGE_MULTI_LABEL_CLASSIFICATION':
    converter = MultiLabelConverter(coco_data)
    write_json_lines(converter, output_jsonl_file_path)
elif task_type == 'OBJECT_DETECTION':
    converter = BoundingBoxConverter(coco_data)
    write_json_lines(converter, output_jsonl_file_path)
elif task_type == 'IMAGE_INSTANCE_SEGMENTATION':
    converter = PolygonConverter(coco_data)
    write_json_lines(converter, output_jsonl_file_path)
else:
    print("ERROR: Invalid Task Type")
    pass

print('Done.')

AML Data Labeling Task type: OBJECT_DETECTION
Converting COCO annotation file "C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco//CustomVisionExport_Coco.json" into JSON Line file "c:\Users\varmag\Documents\AzureML-Playground\COCO-AzureMLDataset\JsonLines\LabeledData.jsonl"
Done.


In [5]:
from azureml.core import Workspace, Dataset

subscription_id = '<subid>'
resource_group = '<resource-group>'
workspace_name = '<workspace>'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='<dataset-name>')
dataset.to_pandas_dataframe()

,image_url,image_details,label
0,workspaceblobstore/customvisionImages/images/0...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Scissors', 'topX': 0.3688725490196..."
1,workspaceblobstore/customvisionImages/images/1...,"{'format': 'jpg', 'width': 900, 'height': 1600}","[{'label': 'Hemlock', 'topX': 0.28555555555555..."
2,workspaceblobstore/customvisionImages/images/2...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Fork', 'topX': 0.18995098039215683..."
3,workspaceblobstore/customvisionImages/images/3...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Scissors', 'topX': 0.4252450980392..."
4,workspaceblobstore/customvisionImages/images/4...,"{'format': 'jpg', 'width': 775, 'height': 1600}","[{'label': 'Hemlock', 'topX': 0.08774193548387..."
5,workspaceblobstore/customvisionImages/images/5...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Scissors', 'topX': 0.3602941176470..."
6,workspaceblobstore/customvisionImages/images/6...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Scissors', 'topX': 0.0416666666666..."
7,workspaceblobstore/customvisionImages/images/7...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Scissors', 'topX': 0.3002450980392..."
8,workspaceblobstore/customvisionImages/images/8...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Scissors', 'topX': 0.1764705882352..."
9,workspaceblobstore/customvisionImages/images/9...,"{'format': 'jpg', 'width': 816, 'height': 612}","[{'label': 'Fork', 'topX': 0.07598039215686274..."
